In [1]:
import pandas as pd
import json
import folium
import os
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm.notebook import tqdm
import time

/tmp/ipykernel_11795/894314210.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [73]:
Susansijang = webdriver.Chrome(service=Service("/home/han/Desktop/Han_ws/00.Data/01.EDA/driver/chromedriver"))
Susansijang.maximize_window()
Susansijang.get("https://www.susansijang.co.kr/nsis/mim/info/mim9030")
Susansijang.implicitly_wait(5)

In [3]:
specie_name_list = Susansijang.find_element(By.ID,"fishing_species")
specie_names = specie_name_list.find_elements(By.TAG_NAME,"option")
species = [specie.text for specie in specie_names]

In [52]:
# get Date of the each types
# Usage : month_box, months = get_date_date(driver_name, "Month")
# month_box : month box location, months : 01, 02, 03, ...
def get_date_data(driver, type) :
    date_group = driver.find_element(By.CLASS_NAME,"form_group")
    type_box = date_group.find_elements(By.ID,"search" + type)[0]
    data_list = type_box.find_elements(By.TAG_NAME,"option")
    data_list = [data.text for data in data_list]
    return type_box, data_list

def next_page(driver) :
    page = driver.find_element(By.CLASS_NAME,"paginate")
    page.find_element(By.CLASS_NAME,"last").click()

# daily auction data
# usage : add_auction_list(driver, list, year, month, day)
# output : [species, origin, size, ... , date]
def add_auction_list(driver, add_list, year, month, day) :
    auction_box = driver.find_element(By.TAG_NAME,"tbody")
    auction_elements = auction_box.find_elements(By.TAG_NAME,"tr")
    auction_rows = [auction_element.text.split(" ") for auction_element in auction_elements]
    [auction_row.append(year+"-"+month+"-"+day) for auction_row in auction_rows]
    add_list.extend(auction_rows)

def get_total_page(driver) :
    page = driver.find_element(By.CLASS_NAME,"paginate")
    return int(page.find_element(By.TAG_NAME,"span").text.split("/")[1])

def get_current_page(driver) :
    page = driver.find_element(By.CLASS_NAME,"paginate")
    return int(page.find_element(By.TAG_NAME,"span").text.split("/")[0])

def click_search_btn(driver) :
    driver.find_element(By.ID,"searchBtn").click()

In [ ]:
date_group = Susansijang.find_element(By.CLASS_NAME,"form_group")

year_box, years = get_date_data(Susansijang,"Year")
month_box, months = get_date_data(Susansijang,"Month")
day_box, days = get_date_data(Susansijang,"Date")

for y in range(-6, -len(years), -1) :

    year_box, years = get_date_data(Susansijang,"Year")
    year_box.send_keys(years[y])
    click_search_btn(Susansijang)
    month_box, months = get_date_data(Susansijang,"Month")


    for m in range(len(months)) :
        month_box, months = get_date_data(Susansijang,"Month")
        month_box.send_keys(months[m])
        click_search_btn(Susansijang)
        day_box, days = get_date_data(Susansijang,"Date")

        for d in range(len(days)) :
            day_box, days = get_date_data(Susansijang,"Date")
            day_box.send_keys(days[d])
            click_search_btn(Susansijang)
            time.sleep(0.2)

            total_page = get_total_page(Susansijang)

            daily_auction_list = []
            for i in range(0, total_page) :
                add_auction_list(Susansijang, daily_auction_list, years[y], months[m], days[d])
                next_page(Susansijang)

            df = pd.DataFrame(daily_auction_list)
            try :
                df = df[df.iloc[:, 9].isna()]
                df.drop(columns=9, inplace=True)
            except :
                pass

            file_path = "./"+years[y]+"/"+months[m]+"/"+years[y]+"-"+months[m]+"-"+days[d]+".csv"
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            df.to_csv(file_path, index=False, header=False)

        time.sleep(0.2)

In [72]:
Susansijang.quit()